# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| export
from collections import defaultdict
from fastcore.all import *
from faststripe.endpoints import eps
from inspect import Parameter, Signature

import httpx, re

In [ ]:
#| hide
import os

In [ ]:
#| exports
stripe_api_url = 'https://api.stripe.com'

In [ ]:
eps[0]


{
    'path': '/v1/account',
    'verb': 'get',
    'op_id': 'GetAccount',
    'summary': 'Retrieve account',
    'params': [
        {
            'name': 'expand',
            'description': 'Specifies which fields in the response should be expanded.'
        }
    ]
}

Now, with each of these descriptions, we can easily create a request that we want on the fly. However, to make it a little bit nicer to use in a library, we'll go ahead and automatically generate functions with proper signatures and docstrings that are then easily accessible in any standard IDE.

In [ ]:
#| export
def _flatten_data(data, prefix=''):
    'Flatten a dictionary of data so that it can be used in a request body.'
    result = {}
    for k,v in data.items():
        key = f'{prefix}[{k}]' if prefix else k
        if isinstance(v, dict): result.update(_flatten_data(v, key))
        elif isinstance(v, list): 
            for i,item in enumerate(v):
                if isinstance(item, dict): result.update(_flatten_data(item, f'{key}[{i}]'))
                else: result[f'{key}[{i}]'] = item
        else: result[key] = v
    return result

In [ ]:
stripe_key = os.environ['STRIPE_SECRET_KEY']
headers = {'Authorization': f'Bearer {stripe_key}'}

In [ ]:
def _mk_func(path, verb, param_info, summary):
    sig_params = [Parameter(param['name'], Parameter.KEYWORD_ONLY, default=None) for param in param_info]
    param_docs = '\n'.join(f"    {param['name']}: {param['description']}" for param in param_info)
    docstring = f"{summary}\n\nParameters:\n{param_docs}" if param_docs else summary
    def m(**kwargs): return getattr(httpx, verb)(stripe_api_url + path, headers=headers, params=_flatten_data(kwargs)).json()
    m.__signature__ = Signature(sig_params)
    m.__doc__ = docstring
    return m

In [ ]:
acc_ep = eps[0]
get_account = _mk_func(acc_ep['path'], acc_ep['verb'], acc_ep['params'], acc_ep['summary'])
get_account??

Signature: get_account(*, expand=None)
Docstring:
Retrieve account

Parameters:
    expand: Specifies which fields in the response should be expanded.
Source:        def m(**kwargs): return getattr(httpx, verb)(stripe_api_url + path, headers=headers, params=_flatten_data(kwargs)).json()
File:      /var/folders/5f/gb9vtfjd68q8pwth3s3t67rr0000gn/T/ipykernel_91319/3787833087.py
Type:      function

In [ ]:
# get_account()

Let's go ahead and design a class since we need to store our API key and for use in the headers for each of these functions. 

In [ ]:
#| export
def _parse_operation_id(op_id):
    'Parse the operation ID to get the resource and verb'
    parts = re.findall(r'[A-Z][a-z]*', op_id)
    verb,*resource_parts = [p.lower() for p in parts]
    r = '_'.join(resource_parts) if resource_parts else 'misc'
    nm = 'create' if verb == 'post' else 'fetch' if verb == 'get' else verb
    return r, nm

For the name of our function, we're going to use this operation ID. However, they use title case so let's make a helper to properly split and construct these names

In [ ]:
r, nm = _parse_operation_id(acc_ep['op_id'])
f'{r}.{nm}'

'account.fetch'

**Note:** We can't set these as the same post/get verbs because when we actually add these as attributes in our class, we're going to use an AttrDict which we cannot overwrite the get method. So I went ahead and changed the names to either fetch or create. 

In [ ]:
#| export
def _mk_func(path, verb, param_info, summary, hdrs={}):
    'Create a function from the Stripe API endpoint'
    sig_params = [Parameter(param['name'], Parameter.KEYWORD_ONLY, default=None) for param in param_info]
    param_docs = '\n'.join(f"    {param['name']}: {param['description']}" for param in param_info)
    docstring = f"{summary}\n\nParameters:\n{param_docs}" if param_docs else summary
    def m(**kwargs): return dict2obj(getattr(httpx, verb)(stripe_api_url + path, headers=hdrs,
                                                          params=_flatten_data(kwargs)).json())
    m.__signature__ = Signature(sig_params)
    m.__doc__ = docstring
    return m

In [ ]:
#| export
class StripeApi:
    def __init__(self, api_key=None, base_url=stripe_api_url):
        self.api_key,self.base_url = api_key,base_url
        self.hdrs = {'Authorization': f'Bearer {self.api_key}'}
        groups = defaultdict(list)
        for ep in eps:
            r, nm = _parse_operation_id(ep['op_id'])
            groups[r].append((ep['path'], ep['verb'], nm, ep['summary'], ep['params']))
        for r, mtds in groups.items():
            group = AttrDict()
            for path, verb, method_name, summary, params in mtds:
                setattr(group, method_name, _mk_func(path, verb, params, summary, hdrs=self.hdrs))
            setattr(self, r, group)

In [ ]:
sapi = StripeApi(api_key=stripe_key)
sapi.account.fetch?

Signature: sapi.account.fetch(*, expand=None)
Docstring:
Retrieve account

Parameters:
    expand: Specifies which fields in the response should be expanded.
File:      /var/folders/5f/gb9vtfjd68q8pwth3s3t67rr0000gn/T/ipykernel_91319/3653438635.py
Type:      function

That is all we need in order to have a fully functional Python SDK that is compliant with the Stripe OpenAPI spec. Kind of insane that in under 100 lines of code, we can get this functionality, which in my opinion is in some respects even better than the official Stripe Python SDK for the simple fact that we can see the parameters that the functions take without looking up the API reference doc online.

Let's go ahead and try to build with this thing. The simplest payment system that you can have in Stripe is a one-time payment URL. Here is how we do this in our new API. First, we have to create a product and its price.

In [ ]:
prod = sapi.products.create(name='Test Product')
prod.id, prod.name

('prod_SQ7qxJf545o8Mx', 'Test Product')

In [ ]:
price = sapi.prices.create(product=prod.id, unit_amount=1000, currency='usd')
price.id, price.unit_amount, price.currency

('price_1RVHaeKGhqIw9PXmUcCcld46', 1000, 'usd')

Now we can create our checkout session with a mode of payment which means that it will only happen once and is not part of any sort of subscription.

In [ ]:
checkout = sapi.checkout_sessions.create(mode='payment', line_items=[dict(price=price.id, quantity=1)],
                                         success_url='https://localhost:5001/success', cancel_url='https://localhost:5001/cancel')
print(f'Payment link: {checkout.url[:64]}...')

Payment link: https://billing.answer.ai/c/pay/cs_test_a1CeqjqiyUvzou1qzPaOSr7T...


Let's make this process even easier by add a higher level api ontop of our StripeApi.

First, let's make it a little bit easier to find an existing products and prices.

In [ ]:
#| export
@patch
def find_product(self:StripeApi, name: str):
    'Find a product by name'
    prods = L(self.products.fetch().data)
    return first(prods, lambda p: p.name == name)

In [ ]:
sapi.find_product('Test Product')

```json
{ 'active': True,
  'attributes': [],
  'created': 1748806240,
  'default_price': None,
  'description': None,
  'id': 'prod_SQ7qxJf545o8Mx',
  'images': [],
  'livemode': False,
  'marketing_features': [],
  'metadata': {},
  'name': 'Test Product',
  'object': 'product',
  'package_dimensions': None,
  'shippable': None,
  'statement_descriptor': None,
  'tax_code': None,
  'type': 'service',
  'unit_label': None,
  'updated': 1748806240,
  'url': None}
```

In [ ]:
#| export
@patch
def find_prices(self:StripeApi, product_id: str):
    'Find all prices associated with a product id'
    return L(self.prices.fetch().data).filter(lambda p: p.product == product_id)

In [ ]:
sapi.find_prices(sapi.find_product('Test Product').id)

[{'id': 'price_1RVHaeKGhqIw9PXmUcCcld46', 'object': 'price', 'active': True, 'billing_scheme': 'per_unit', 'created': 1748806240, 'currency': 'usd', 'custom_unit_amount': None, 'livemode': False, 'lookup_key': None, 'metadata': {}, 'nickname': None, 'product': 'prod_SQ7qxJf545o8Mx', 'recurring': None, 'tax_behavior': 'unspecified', 'tiers_mode': None, 'transform_quantity': None, 'type': 'one_time', 'unit_amount': 1000, 'unit_amount_decimal': '1000'}]

In [ ]:
#| export
@patch
def priced_product(self:StripeApi, product_name, amount_cents, currency='usd', recurring=None, description=None):
    "Create a product and price if they don't exist"
    prod_params = dict(name=product_name)
    if description: prod_params['description'] = description
    prod = self.find_product(product_name) or self.products.create(**prod_params)
    price_params = dict(product=prod.id, unit_amount=amount_cents, currency=currency)
    if recurring: price_params['recurring'] = recurring
    price = first(self.find_prices(prod.id)) or self.prices.create(**price_params)
    return prod, price

In [ ]:
prod, price = sapi.priced_product('Test Product', 1000, 'usd')
prod.name, price.id, price.unit_amount

('Test Product', 'price_1RVHaeKGhqIw9PXmUcCcld46', 1000)

Now we can automatically create a product or use an existing one when create a one time payment link.

In [ ]:
#| export
@patch
def one_time_payment(self:StripeApi, product_name, amount_cents, success_url, cancel_url, currency='usd', **kw):
    'Create a simple one-time payment checkout'
    _, price = self.priced_product(product_name, amount_cents, currency)
    return self.checkout_sessions.create(mode='payment', line_items=[dict(price=price.id, quantity=1)],
                                         automatic_tax={'enabled': True}, success_url=success_url, cancel_url=cancel_url, **kw)

In [ ]:
checkout = sapi.one_time_payment('Test Product', 1000, 'https://localhost:5001/success', 'https://localhost:5001/cancel', 'usd')
print(f'Payment link: {checkout.url[:64]}...')

Payment link: https://billing.answer.ai/c/pay/cs_test_a12wY9kQ1i3R5AWGahSrl0tK...


Another common use case is subscriptions. Let's go ahead and create a version that makes subscription creation just as easy.

In [ ]:
#| export
@patch
def subscription(self:StripeApi, product_name, amount_cents, success_url, cancel_url,
                 currency='usd', interval='month', **kw):
    'Create a simple recurring subscription'
    _, price = self.priced_product(product_name, amount_cents, currency, recurring=dict(interval=interval))
    return self.checkout_sessions.create(mode='subscription', success_url=success_url, cancel_url=cancel_url,
                                         line_items=[dict(price=price.id, quantity=1)], **kw)

In [ ]:
sub_checkout = sapi.subscription('Test Subscription Product', 1000, 'https://localhost:5001/success', 'https://localhost:5001/cancel')
print(f'Payment link: {sub_checkout.url[:64]}...')

Payment link: https://billing.answer.ai/c/pay/cs_test_a1Vu4oxCVP4E9HZ7FAH7g4UZ...


**Note:** You'll want to use Stripe's webhook functionality for detecting payment and subscription events. To do so, you'll utilize the Python Stripe SDK that the event actually came from stripe.

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()